<a href="https://colab.research.google.com/github/julieisok/OurFisrtML/blob/master/3_%ED%94%BC%EB%B2%97%ED%85%8C%EC%9D%B4%EB%B8%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 숨겨진 정보를 추출

- 데이터를 들여다 볼때, 특정 컬럼에서 반복적인 데이터들이 나타난다. 그런데 이런 증상이 여러 컬럼에 걸쳐서 보인다. 
- 통찰
  - 데이터의 빈도를 기준으로 표현 (차후) 
  - 반복 데이터를 기준으로 데이터를 재정립 (인덱스, 컬럼, 값 등을 재구성) 
     - 피벗 테이블 


In [0]:
import numpy as np
import pandas as pd

In [2]:
# 데이터 로드
df = pd.read_excel('/content/sales.xlsx')
df.head(10)

,Account,Name,Rep,Manager,Product,Quantity,Price,Status
0,714466,Trantow-Barrows,Craig Booker,Debra Henley,CPU,1,30000,presented
1,714466,Trantow-Barrows,Craig Booker,Debra Henley,Software,1,10000,presented
2,714466,Trantow-Barrows,Craig Booker,Debra Henley,Maintenance,2,5000,pending
3,737550,"Fritsch, Russel and Anderson",Craig Booker,Debra Henley,CPU,1,35000,declined
4,146832,Kiehn-Spinka,Daniel Hilton,Debra Henley,CPU,2,65000,won
5,218895,Kulas Inc,Daniel Hilton,Debra Henley,CPU,2,40000,pending
6,218895,Kulas Inc,Daniel Hilton,Debra Henley,Software,1,10000,presented
7,412290,Jerde-Hilpert,John Smith,Debra Henley,Maintenance,2,5000,pending
8,740150,Barton LLC,John Smith,Debra Henley,CPU,1,35000,declined
9,141962,Herman LLC,Cedric Moss,Fred Anderson,CPU,2,65000,won


In [3]:
# 특정 컬럼에서 사용되는 데이터가 어떤것들이 있는가?
# Manager 데이터만 출력하시오 
df.columns
#df.Manager
#df['Manager']
#df.loc[ : , ['Manager']]
df.iloc[:, 3]

0      Debra Henley
1      Debra Henley
2      Debra Henley
3      Debra Henley
4      Debra Henley
5      Debra Henley
6      Debra Henley
7      Debra Henley
8      Debra Henley
9     Fred Anderson
10    Fred Anderson
11    Fred Anderson
12    Fred Anderson
13    Fred Anderson
14    Fred Anderson
15    Fred Anderson
16    Fred Anderson
Name: Manager, dtype: object

In [4]:
# 중복을 제거하고 매니저 이름만 출력
df[ 'Manager' ].unique()

array(['Debra Henley', 'Fred Anderson'], dtype=object)

In [5]:
df[ 'Rep' ].unique(), len(df[ 'Rep' ].unique())

(array(['Craig Booker', 'Daniel Hilton', 'John Smith', 'Cedric Moss',
        'Wendy Yule'], dtype=object), 5)

In [6]:
df[ 'Name' ].unique(), len(df[ 'Name' ].unique())

(array(['Trantow-Barrows', 'Fritsch, Russel and Anderson', 'Kiehn-Spinka',
        'Kulas Inc', 'Jerde-Hilpert', 'Barton LLC', 'Herman LLC',
        'Purdy-Kunde', 'Stokes LLC', 'Kassulke, Ondricka and Metz',
        'Keeling LLC', 'Koepp Ltd'], dtype=object), 12)

- 이름으로 예상되는 3개의 컬럼을 살펴보니 조직관계라는 숨겨진 특성이 보인다
- 3개의 컬럼은 중복적으로 데이터가 노출된다 

In [7]:
df[ 'Product' ].unique(), len(df[ 'Product' ].unique())

(array(['CPU', 'Software', 'Maintenance', 'Monitor'], dtype=object), 4)

In [8]:
# 데이터의 크기
df.shape

(17, 8)

In [9]:
# 하나를 기준으로 그냥 돌려보겠다 
# name 컬럼을 중심으로 피벗을 수행하겠다
tmp = pd.pivot_table( df, index=['Name'] )
tmp.shape
# 기본적으로 수치만 데이터로 남고, 수치들은 산술 평균으로 처리되어 있다
# API의 기본 처리함수(매개변수) aggfunc가 mean으로 설정되어 있다  

(12, 3)

In [11]:
# 오직 수치만 pivot_table에 디폴트로 설정된 aggrFunc=mean에 의해 평균값으로 산출됨 
tmp[:2]


,Account,Price,Quantity
Name,,,
Barton LLC,740150,35000,1.0
"Fritsch, Russel and Anderson",737550,35000,1.0


In [12]:
# 인덱스를 나열해 보았더니, 인덱스가 여러개 생겼다 
tmp = pd.pivot_table( df, index=['Name', 'Rep', 'Manager'] )
tmp

,,,Account,Price,Quantity
Name,Rep,Manager,,,
Barton LLC,John Smith,Debra Henley,740150,35000,1.000000
"Fritsch, Russel and Anderson",Craig Booker,Debra Henley,737550,35000,1.000000
Herman LLC,Cedric Moss,Fred Anderson,141962,65000,2.000000
Jerde-Hilpert,John Smith,Debra Henley,412290,5000,2.000000
"Kassulke, Ondricka and Metz",Wendy Yule,Fred Anderson,307599,7000,3.000000
Keeling LLC,Wendy Yule,Fred Anderson,688981,100000,5.000000
Kiehn-Spinka,Daniel Hilton,Debra Henley,146832,65000,2.000000
Koepp Ltd,Wendy Yule,Fred Anderson,729833,35000,2.000000
Kulas Inc,Daniel Hilton,Debra Henley,218895,25000,1.500000


In [16]:
# 레벨은(깊이는) 3개가 생겼다 -> 데이터가 반복적으로 등장한다 
len(tmp.index), tmp.index, len(tmp.index.levels)

(12,
 MultiIndex([(                  'Barton LLC',    'John Smith',  'Debra Henley'),
             ('Fritsch, Russel and Anderson',  'Craig Booker',  'Debra Henley'),
             (                  'Herman LLC',   'Cedric Moss', 'Fred Anderson'),
             (               'Jerde-Hilpert',    'John Smith',  'Debra Henley'),
             ( 'Kassulke, Ondricka and Metz',    'Wendy Yule', 'Fred Anderson'),
             (                 'Keeling LLC',    'Wendy Yule', 'Fred Anderson'),
             (                'Kiehn-Spinka', 'Daniel Hilton',  'Debra Henley'),
             (                   'Koepp Ltd',    'Wendy Yule', 'Fred Anderson'),
             (                   'Kulas Inc', 'Daniel Hilton',  'Debra Henley'),
             (                 'Purdy-Kunde',   'Cedric Moss', 'Fred Anderson'),
             (                  'Stokes LLC',   'Cedric Moss', 'Fred Anderson'),
             (             'Trantow-Barrows',  'Craig Booker',  'Debra Henley')],
            names=['Na

In [17]:
# 인덱스를 본래 의미(서열관계)에 맞춰서 재배치 
# 인덱스를 정확하게 배치함으로써 조직도를 알수 있게 되었다 
tmp = pd.pivot_table( df, index=['Manager','Rep',  'Name'] )
tmp

Account  ...  Quantity
Manager       Rep           Name                                   ...          
Debra Henley  Craig Booker  Fritsch, Russel and Anderson   737550  ...  1.000000
                            Trantow-Barrows                714466  ...  1.333333
              Daniel Hilton Kiehn-Spinka                   146832  ...  2.000000
                            Kulas Inc                      218895  ...  1.500000
              John Smith    Barton LLC                     740150  ...  1.000000
                            Jerde-Hilpert                  412290  ...  2.000000
Fred Anderson Cedric Moss   Herman LLC                     141962  ...  2.000000
                            Purdy-Kunde                    163416  ...  1.000000
                            Stokes LLC                     239344  ...  1.000000
              Wendy Yule    Kassulke, Ondricka and Metz    307599  ...  3.000000
                            Keeling LLC                    688981  ...  5.000000
                            Koepp Ltd                      729833  ...  2.000000

[12 rows x 3 columns]

In [20]:
# 값을 지정함으로써 컬럼이 자동으로 선택된다 
tmp = pd.pivot_table( df, index=['Manager','Rep'], values=['Price'] )
tmp

Price
Manager       Rep                        
Debra Henley  Craig Booker   20000.000000
              Daniel Hilton  38333.333333
              John Smith     20000.000000
Fred Anderson Cedric Moss    27500.000000
              Wendy Yule     44250.000000

In [22]:
# 위의 테이블 상태에서 산술평균이 아닌, 판매 총액을 보고 싶다 
tmp = pd.pivot_table( df, index=['Manager','Rep'], values=['Price'], aggfunc=np.sum )
tmp

Price
Manager       Rep                  
Debra Henley  Craig Booker    80000
              Daniel Hilton  115000
              John Smith      40000
Fred Anderson Cedric Moss    110000
              Wendy Yule     177000

In [24]:
tmp = pd.pivot_table( df, index=['Manager','Rep'], values=['Price', 'Quantity'], aggfunc= [np.sum, len, np.mean] )
tmp

# 판매 건수가 len으로 처리된 것임 
# 판매 된 물품별 건수는 Quantity로 확인이 가능하다 

sum           ...          mean          
                              Price Quantity  ...         Price  Quantity
Manager       Rep                             ...                        
Debra Henley  Craig Booker    80000        5  ...  20000.000000  1.250000
              Daniel Hilton  115000        5  ...  38333.333333  1.666667
              John Smith      40000        3  ...  20000.000000  1.500000
Fred Anderson Cedric Moss    110000        5  ...  27500.000000  1.250000
              Wendy Yule     177000       12  ...  44250.000000  3.000000

[5 rows x 6 columns]

In [25]:
# mean price의 값을 0-1사이로 정규화해서 새로운 컬럼 sell_rate를 추가하시오 
# 최대값을 1로 두고 배치

tmp.columns

MultiIndex([( 'sum',    'Price'),
            ( 'sum', 'Quantity'),
            ( 'len',    'Price'),
            ( 'len', 'Quantity'),
            ('mean',    'Price'),
            ('mean', 'Quantity')],
           )

In [29]:
tmp[ tmp.columns[-2] ]

Manager        Rep          
Debra Henley   Craig Booker     20000.000000
               Daniel Hilton    38333.333333
               John Smith       20000.000000
Fred Anderson  Cedric Moss      27500.000000
               Wendy Yule       44250.000000
Name: (mean, Price), dtype: float64

In [34]:
# 컬럼명을 재구성
# zip 함수에 인자에 *(포인터)를 붙이면 -단, 데이터가 연속형일경우- 
# 각 포지션 별로 데이터를 모아서 멤버로 구성한다 

fs, bs = list(zip(*tmp.columns))
fs
bs

('Price', 'Quantity', 'Price', 'Quantity', 'Price', 'Quantity')

In [35]:
# list(set(fs)) : 원재료를 중복 제거하는 것 => 단, 순서가 보장되지는 않는다 
# 따라서 데이터가 소수이면 직접 타이핑 하는 것이 효율적임. 데이터에 따라 다른 로직으로 접근해야한다. 
[ '%s_%s' % (f, b) for f in list(set(fs)) for b in list(set(bs)) ]

['len_Quantity',
 'len_Price',
 'mean_Quantity',
 'mean_Price',
 'sum_Quantity',
 'sum_Price']

In [0]:
# 컬럼명 재구성 : level2 => level1 대체 
tmp.columns = [ 'sum_Price', 'sum_Quantity', 'len_Price', 'len_Quantity', 'mean_Price', 'mean_Quantity'] 

In [37]:
# 대상
print( tmp['mean_Price'] )

Manager        Rep          
Debra Henley   Craig Booker     20000.000000
               Daniel Hilton    38333.333333
               John Smith       20000.000000
Fred Anderson  Cedric Moss      27500.000000
               Wendy Yule       44250.000000
Name: mean_Price, dtype: float64


In [38]:
# mean price의 값을 0-1 사이로 정규화해서 새로운 컬럼 sell_rate를 추가하시오
# 최대값 기준
tmp[ 'mean_Price' ].max()

44250.0

In [40]:
# 벡터 대 스칼라 연산
a = tmp['mean_Price'] / tmp['mean_Price'].max()
a

Manager        Rep          
Debra Henley   Craig Booker     0.451977
               Daniel Hilton    0.866290
               John Smith       0.451977
Fred Anderson  Cedric Moss      0.621469
               Wendy Yule       1.000000
Name: mean_Price, dtype: float64

In [42]:
tmp['sell_rate'] = a
tmp.head(2)

sum_Price  sum_Quantity  ...  mean_Quantity  sell_rate
Manager      Rep                                     ...                          
Debra Henley Craig Booker       80000             5  ...       1.250000   0.451977
             Daniel Hilton     115000             5  ...       1.666667   0.866290

[2 rows x 7 columns]

In [46]:
# 제품별로 판매 사항을 확인
# 컬럼이 3 levels (aggfunc * columns.unique * values)
tmp2 = pd.pivot_table( df, 
                       index = ['Manager', 'Rep'], 
                       values = ['Price'], 
                       columns = ['Product'], 
                       aggfunc = [np.sum])
tmp2

sum                             
                                Price                             
Product                           CPU Maintenance Monitor Software
Manager       Rep                                                 
Debra Henley  Craig Booker    65000.0      5000.0     NaN  10000.0
              Daniel Hilton  105000.0         NaN     NaN  10000.0
              John Smith      35000.0      5000.0     NaN      NaN
Fred Anderson Cedric Moss     95000.0      5000.0     NaN  10000.0
              Wendy Yule     165000.0      7000.0  5000.0      NaN

In [47]:
# NaN 후처리
tmp2 = pd.pivot_table( df, 
                       index = ['Manager', 'Rep'], 
                       values = ['Price'], 
                       columns = ['Product'], 
                       aggfunc = [np.sum]).fillna(0)
tmp2

sum                             
                                Price                             
Product                           CPU Maintenance Monitor Software
Manager       Rep                                                 
Debra Henley  Craig Booker    65000.0      5000.0     0.0  10000.0
              Daniel Hilton  105000.0         0.0     0.0  10000.0
              John Smith      35000.0      5000.0     0.0      0.0
Fred Anderson Cedric Moss     95000.0      5000.0     0.0  10000.0
              Wendy Yule     165000.0      7000.0  5000.0      0.0

In [48]:
# NaN 전처리 => aggfunc에 따라 달라지겠지만, 원재료의 타입을 따라간다 
tmp2 = pd.pivot_table( df, 
                       index = ['Manager', 'Rep'], 
                       values = ['Price'], 
                       columns = ['Product'], 
                       aggfunc = [np.sum], 
                      fill_value = 0
                      )
tmp2

sum                             
                              Price                             
Product                         CPU Maintenance Monitor Software
Manager       Rep                                               
Debra Henley  Craig Booker    65000        5000       0    10000
              Daniel Hilton  105000           0       0    10000
              John Smith      35000        5000       0        0
Fred Anderson Cedric Moss     95000        5000       0    10000
              Wendy Yule     165000        7000    5000        0

In [51]:
# 데이터 관점을 총액과 수량, 물품을 같은 (관리부서별) 비중으로 본다면
# 매출이 없는 부분은 아예 빠진다 => 인덱스로 위치하게 되면 
tmp2 = pd.pivot_table( df, 
                       index = ['Manager', 'Rep', 'Product'], 
                       values = ['Price', 'Quantity'],
                       aggfunc = [np.sum], 
                       fill_value = 0
                      )
tmp2

sum         
                                          Price Quantity
Manager       Rep           Product                     
Debra Henley  Craig Booker  CPU           65000        2
                            Maintenance    5000        2
                            Software      10000        1
              Daniel Hilton CPU          105000        4
                            Software      10000        1
              John Smith    CPU           35000        1
                            Maintenance    5000        2
Fred Anderson Cedric Moss   CPU           95000        3
                            Maintenance    5000        1
                            Software      10000        1
              Wendy Yule    CPU          165000        7
                            Maintenance    7000        3
                            Monitor        5000        2